In [1]:
import pandas as pd
import numpy as np

from mltracker import ExperimentTracker

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Data Creation

In [3]:
df = pd.DataFrame({

  'a':[1,2,3,4,5,6,7],
  'b':[11,22,33,44,55,66,77],
  'c':np.random.randint(0,10,7)
})

df['y'] = df.a+df.b*df.c
df

,a,b,c,y
0,1,11,8,89
1,2,22,0,2
2,3,33,3,102
3,4,44,7,312
4,5,55,5,280
5,6,66,4,270
6,7,77,7,546


In [4]:
df_data = df.drop(columns='y')
df_target = df.y

X_train, X_test, y_train, y_test = train_test_split(df_data, df_target)


# ML FLOW

In [5]:
from mltracker import ExperimentTracker

from sklearn.ensemble import RandomForestRegressor


experiment_name = 'Regression'


tracker = ExperimentTracker(
    tracking_platform="mlflow",
    experiment_name=experiment_name,
    tracking_uri="http://localhost:5000"
)

# # Example usage with Weights & Biases
# tracker = ExperimentTracker(
#     tracking_platform="wandb",
#     project_name="my_project",
#     experiment_name=experiment_name
# )

# Both can be used with the same interface
with tracker:
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)
    
    tracker.log_metric( )
    tracker.log_params( )

2024/11/05 12:04:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run run_20241105_120416 at: http://localhost:5000/#/experiments/960292128128523123/runs/673f722e938b4aa4b0cee1ff610f0635.
2024/11/05 12:04:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/960292128128523123.
